### Preparation

#### 1.1 Import library

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import polars as pl

from src.lib.etl.class_ETL import ETL
from src.lib.frc_model.class_Forecast_Model import Forecast_Model

pd.options.display.float_format = '{:,}'.format
pd.options.display.max_columns = 100

#### 1.2 Create udf - user defined function

In [2]:
def filter_data_input(df, frc_value, cond):
    
    # fitler your df
    if cond:
        for _key, _values in cond.items():
            df = df[df[_key] == _values ]
    
    df_pivot = df.groupby(['complete_date'],as_index = False)[frc_value].sum()
    df_pivot.sort_values('complete_date',inplace= True)

    return df_pivot

### Forecast

In [3]:
# call triendt to handle import, export data
triendt = ETL(credentials_file_path= './../../config/cred.json',
              local_database= './data/database/fa_database.db'
              )

In [5]:
# df_bigquery_data = triendt.extract_bigquery_data(start_date= '2024-05-07', table_name= 'fct_fa_pnl_mini')
# df_bigquery_data

##### 2.1 Check if historical data exists 
* If your local database is not update data yet. Just use function auto_updat_data above

In [4]:
df_fa_pnl_mini = triendt.extract_local_data(table_name='fct_fa_pnl_mini')
df_fa_pnl_mini.complete_date = pd.to_datetime(df_fa_pnl_mini.complete_date, format='mixed')
df_fa_pnl_mini.tail()

In [7]:
## to delete all data and update whole table, choose update_type = 'update_as_new'
triendt.auto_update_data(table_name='fct_fa_pnl_mini', update_type='append')

##### 2.2 Input filter condition below and change your df name with condition.

In [15]:
filter_conditions = {
                    # "province": "han",
                     "service_type": "truck",
                     }

# filter_conditions = None

df_historical_data = filter_data_input(df= df_fa_pnl_mini,frc_value= 'gsv_excluded_vat', cond= filter_conditions)

### call forecast model
forecast_gsv_bike_sgn = Forecast_Model(historical_data=df_historical_data,
                                      date_column='complete_date',
                                      forecast_value= 'gsv_excluded_vat',
                                      )

### forecast for the next 365 days
df_forecast = forecast_gsv_bike_sgn.call_forecast_model(no_of_days_forecast= 365)
df_forecast.tail()

In [16]:
df= forecast_gsv_bike_sgn.resample_and_plot_data(_df_forecast=df_forecast.copy(),start_date='2022-01-01')


### Export data
* call funtion export_data, provide dataframe, file type (excel or csv) and file name

In [17]:
# forecast_gsv_bike_sgn.export_data(df=df_forecast,file_name='gsv_bike_sgn',file_type='excel')
forecast_gsv_bike_sgn.export_data(df=df_forecast,file_name='gsv_truck_24',file_type='excel')